In [1]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tqdm import tqdm
import torchvision
from dataset import make_datapath_list, VideoTransform, get_label_id_dictionary, VideoDataset

/home/vlsimin95/anaconda3/envs/torch38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_root_path = '../data/UCF-101/train/'
train_path_list = make_datapath_list(train_root_path)
val_root_path = '../data/UCF-101/val/'
val_path_list = make_datapath_list(val_root_path)

resize, crop_size =224, 224
mean, std = [103, 117, 123], [1, 1, 1]
video_transform = VideoTransform(resize, crop_size, mean, std)

/home/vlsimin95/anaconda3/envs/torch38/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [3]:
id_label_dict, label_id_dict= get_label_id_dictionary()
print(label_id_dict)

{'ApplyEyeMakeup': 0, 'ApplyLipstick': 1, 'Archery': 2, 'BabyCrawling': 3, 'BalanceBeam': 4}


In [4]:
train_dataset = VideoDataset(train_path_list, label_id_dict, num_segments=16, phase='train', transform=video_transform)
val_dataset = VideoDataset(val_path_list, label_id_dict, num_segments=16, phase='val', transform=video_transform)

In [5]:
batch_size = 1
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
whole_dataloader = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

In [6]:
from model.eco_lite import ECO_Lite
my_model = ECO_Lite(num_of_classes=5)
my_loss = torch.nn.CrossEntropyLoss()
my_opt = torch.optim.SGD(my_model.parameters(),lr=0.001, momentum=0.9, dampening=0, nesterov=True)

In [7]:
index = 0
print(train_dataset.__getitem__(index)[0].shape)
print(train_dataset.__getitem__(index)[1])
print(train_dataset.__getitem__(index)[2])
print(train_dataset.__getitem__(index)[3])

torch.Size([16, 3, 224, 224])
BalanceBeam
4
../data/UCF-101/train/BalanceBeam/v_BalanceBeam_g19_c03


In [8]:
index = 0
print(val_dataset.__getitem__(index)[0].shape)
print(val_dataset.__getitem__(index)[1])
print(val_dataset.__getitem__(index)[2])
print(val_dataset.__getitem__(index)[3])

torch.Size([16, 3, 224, 224])
BalanceBeam
4
../data/UCF-101/val/BalanceBeam/v_BalanceBeam_g23_c05


In [9]:
batch_iterator = iter(val_dataloader)
imgs_transformeds, labels, label_ids, dir_path = next(batch_iterator)
print(imgs_transformeds.shape)
print(label_ids)

torch.Size([1, 16, 3, 224, 224])
tensor([4])


In [10]:
from train import train_model
train_model(my_model, whole_dataloader, my_loss, my_opt, 100)

device = cuda:0
load model to cuda:0
EPOCH 1/100
------------


100%|██████████| 358/358 [00:21<00:00, 16.30it/s]


val Loss: 1.6088 Acc: 0.2235
EPOCH 2/100
------------


100%|██████████| 930/930 [02:51<00:00,  5.42it/s]


train Loss: 1.5847 Acc: 0.2667


...

100%|██████████| 930/930 [02:58<00:00,  5.21it/s]


train Loss: 0.9862 Acc: 0.5710


100%|██████████| 358/358 [00:24<00:00, 14.59it/s]


val Loss: 7.9030 Acc: 0.0726
EPOCH 7/100
------------


 55%|█████▌    | 513/930 [01:40<01:21,  5.11it/s]